In [ ]:
!pip install -U sacremoses  # For bioGPT BPE Tokenizer

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import re
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.auto import tqdm

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "/content/gdrive/MyDrive/"
os.listdir(path)
data = pd.read_csv(path+'comback_rev.csv')
data.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,text,readmission_30
0,Subdural hematoma ETOH abuse Wernicke's ence...,1.0
1,Primary:eosinophilic enteropathy on IL-5 inhib...,1.0
2,PROSTATE CANCER,1.0
3,clotted brachiocephalic fistulaesrdhtnDM II,1.0
4,DyspneaAsthmaCOPDCHFDOE,1.0


In [ ]:
def extract_and_combine_conditions(text):
    parts = text.split('Secondary:')
    primary_conditions = parts[0].replace('Primary:', '').strip()
    secondary_conditions = parts[1].strip() if len(parts) > 1 else ''
    combined_conditions = primary_conditions + ', ' + secondary_conditions if secondary_conditions else primary_conditions
    return combined_conditions

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\b(patientname|patientid):\s*\S+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = extract_and_combine_conditions(text)
    return text

In [ ]:
class ReadmissionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(self.encodings[key][idx]) for key in self.encodings}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
data['clean_text']=data['text'].apply(clean_text)

In [ ]:
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(
    data['clean_text'], data['readmission_30'], test_size=0.2, random_state=42)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_val_texts, train_val_labels, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/biogpt')

In [ ]:
# 토크나이저 사용
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

In [ ]:
# 데이터셋 생성
train_dataset = ReadmissionDataset(train_encodings, train_labels.tolist())
val_dataset = ReadmissionDataset(val_encodings, val_labels.tolist())
test_dataset = ReadmissionDataset(test_encodings, test_labels.tolist())

In [ ]:
# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('microsoft/biogpt', num_labels=data['readmission_30'].nunique())

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

## earlyStopping

In [ ]:
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss

        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0

        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
import torch

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            loss = criterion(logits, labels)
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    average_loss = total_loss / len(test_loader)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')

    return average_loss, f1, recall, accuracy, precision


In [ ]:
early_stopper = EarlyStopping(patience=10, min_delta=0.001)
epoch = 50
for epoch in range(epoch):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}', leave=False)

    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = criterion(logits, labels)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})

    average_train_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch} finished - Avg Train Loss: {average_train_loss:.4f}')

    train_loss, train_f1, train_recall, train_acc, train_prec = evaluate(model, train_loader, criterion, device)
    print(f'Train - Loss: {train_loss:.4f}, F1 Score: {train_f1:.4f}, Recall: {train_recall:.4f}, Accuracy: {train_acc:.4f}, Precision: {train_prec:.4f}')
    val_loss, val_f1, val_recall, val_acc, val_prec = evaluate(model, val_loader, criterion, device)
    print(f'Validation - Loss: {val_loss:.4f}, F1 Score: {val_f1:.4f}, Recall: {val_recall:.4f}, Accuracy: {val_acc:.4f}, Precision: {val_prec:.4f}')

    test_loss, test_f1, test_recall, test_acc, test_prec = evaluate(model, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, F1 Score: {test_f1:.4f}, Recall: {test_recall:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}')

    model.save_pretrained(f'/content/gdrive/MyDrive/help/gpt_vali_sgd_0.0001_epoch_{epoch}_val_loss_{val_loss}/test/bioclinical-bert-readmission')
    model.config.to_json_file(f'/content/gdrive/MyDrive/help/gpt_vali_sgd_0.0001_epoch_{epoch}_val_loss_{val_loss}/config.json')
    tokenizer.save_pretrained(f'/content/gdrive/MyDrive/help/gpt_vali_sgd_0.0001_epoch_{epoch}_val_loss_{val_loss}/test/bioclinical-bert-readmission')
    print("\n")
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print("Early stopping triggered")
        break

Epoch 1:   0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 0 finished - Avg Train Loss: 0.6529
Train - Loss: 0.6242, F1 Score: 0.5950, Recall: 0.6677, Accuracy: 0.6677, Precision: 0.6174
Validation - Loss: 0.6199, F1 Score: 0.6045, Recall: 0.6748, Accuracy: 0.6748, Precision: 0.6230
Test Loss: 0.6237, F1 Score: 0.6005, Recall: 0.6710, Accuracy: 0.6710, Precision: 0.6229




Epoch 2:   0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 1 finished - Avg Train Loss: 0.6242
Train - Loss: 0.6145, F1 Score: 0.6104, Recall: 0.6733, Accuracy: 0.6733, Precision: 0.6321
Validation - Loss: 0.6125, F1 Score: 0.6182, Recall: 0.6789, Accuracy: 0.6789, Precision: 0.6348
Test Loss: 0.6154, F1 Score: 0.6149, Recall: 0.6757, Accuracy: 0.6757, Precision: 0.6352




Epoch 3:   0%|          | 0/3750 [00:00<?, ?it/s]

...

Epoch 22 finished - Avg Train Loss: 0.5905
Train - Loss: 0.5852, F1 Score: 0.6623, Recall: 0.6972, Accuracy: 0.6972, Precision: 0.6739
Validation - Loss: 0.5981, F1 Score: 0.6503, Recall: 0.6863, Accuracy: 0.6863, Precision: 0.6547
Test Loss: 0.6024, F1 Score: 0.6447, Recall: 0.6814, Accuracy: 0.6814, Precision: 0.6506




Epoch 24:   0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 23 finished - Avg Train Loss: 0.5896
Train - Loss: 0.5847, F1 Score: 0.6640, Recall: 0.6978, Accuracy: 0.6978, Precision: 0.6749
Validation - Loss: 0.5984, F1 Score: 0.6500, Recall: 0.6848, Accuracy: 0.6848, Precision: 0.6533
Test Loss: 0.6029, F1 Score: 0.6438, Recall: 0.6797, Accuracy: 0.6797, Precision: 0.6487




Epoch 25:   0%|          | 0/3750 [00:00<?, ?it/s]

Epoch 24 finished - Avg Train Loss: 0.5889
Train - Loss: 0.5822, F1 Score: 0.6371, Recall: 0.6975, Accuracy: 0.6975, Precision: 0.6818
Validation - Loss: 0.5952, F1 Score: 0.6325, Recall: 0.6925, Accuracy: 0.6925, Precision: 0.6615
Test Loss: 0.6004, F1 Score: 0.6259, Recall: 0.6875, Accuracy: 0.6875, Precision: 0.6585


Early stopping triggered


In [ ]:
test_loss, test_f1, test_recall, test_acc, test_prec = evaluate(model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f}, F1 Score: {test_f1:.4f}, Recall: {test_recall:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}')

Test Loss: 0.6004, F1 Score: 0.6259, Recall: 0.6875, Accuracy: 0.6875, Precision: 0.6585


In [ ]:
model.save_pretrained('/content/gdrive/MyDrive/help/gpt_vali_adam_final/test/bioclinical-bert-readmission')
model.config.to_json_file('/content/gdrive/MyDrive/help/gpt_vali_adam_final/config.json')
tokenizer.save_pretrained('/content/gdrive/MyDrive/help/gpt_vali_adam_final/test/bioclinical-bert-readmission')
!tar -cvf bioclinical-bert_finetuned test/
!mv ./bioclinical-bert_finetuned.tar.gz /content/drive/MyDrive/bioclinical-bert_finetuned.tar.gz

tar: test: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors
mv: cannot stat './bioclinical-bert_finetuned.tar.gz': No such file or directory
